In [1]:
import rasterio
import numpy as np
import os
import requests
import skfuzzy as fuzz
from skfuzzy import control as ctrl
from rasterio.transform import from_bounds

# ------------------------------------------------------------
# Folders
data_in_geoserver_folder = "../geoserver_data/data"
os.makedirs(data_in_geoserver_folder, exist_ok=True)

# ------------------------------------------------------------
# WMS parameters
pm10_wms_url = "https://wms.zh.ch/AwelLHPM10JahreZHWMS"
year = 2023
layer_name = f"pm10-jahre-{year}"
bbox = (2654500, 1222400, 2720000, 1300000)
width, height = 1600, 1200

params = {
    "SERVICE": "WMS",
    "VERSION": "1.3.0",
    "REQUEST": "GetMap",
    "LAYERS": layer_name,
    "CRS": "EPSG:2056",
    "BBOX": f"{bbox[0]},{bbox[1]},{bbox[2]},{bbox[3]}",
    "WIDTH": width,
    "HEIGHT": height,
    "FORMAT": "image/tiff"
}

# Download TIFF
r = requests.get(pm10_wms_url, params=params)
r.raise_for_status()
raw_tif_path = os.path.join(data_in_geoserver_folder, "zurich_pm10_raw.tif")
with open(raw_tif_path, "wb") as f:
    f.write(r.content)
print(f"✅ Raw PM10 TIFF downloaded: {raw_tif_path}")

# ------------------------------------------------------------
# Read the TIFF
with rasterio.open(raw_tif_path) as src:
    arr = src.read(1)
    transform = src.transform
    crs = src.crs

# ------------------------------------------------------------
# Mask for valid pixels
mask = arr != 255

# Normalize all pixels to data_max
data_max = 16
arr_norm = np.zeros_like(arr, dtype=float)
arr_norm[mask] = (arr[mask] / arr[mask].max()) * data_max
arr_norm[~mask] = np.nan  # keep no-data as NaN

# ------------------------------------------------------------
# Fuzzy logic setup
low_threshold = 15
high_threshold = 25
fuzzy_max = 40

# Antecedent
pm10 = ctrl.Antecedent(np.linspace(0, fuzzy_max, 100), 'pm10')
pm10['good'] = fuzz.trimf(pm10.universe, [0, 0, low_threshold])
pm10['moderate'] = fuzz.trimf(pm10.universe, [low_threshold, (low_threshold + high_threshold)/2, high_threshold])
pm10['bad'] = fuzz.trimf(pm10.universe, [high_threshold, fuzzy_max, fuzzy_max])

# Consequent
quality = ctrl.Consequent(np.linspace(0, 100, 100), 'quality')
quality['very_good'] = fuzz.trimf(quality.universe, [0, 0, 25])
quality['good'] = fuzz.trimf(quality.universe, [20, 40, 60])
quality['liveable'] = fuzz.trimf(quality.universe, [50, 60, 70])
quality['bad'] = fuzz.trimf(quality.universe, [65, 80, 90])
quality['very_bad'] = fuzz.trimf(quality.universe, [85, 100, 100])

# Rules
rules = [
    ctrl.Rule(pm10['bad'], quality['very_bad']),
    ctrl.Rule(pm10['moderate'], quality['liveable']),
    ctrl.Rule(pm10['good'], quality['very_good']),
]

quality_ctrl = ctrl.ControlSystem(rules)

# ------------------------------------------------------------
# Pixel-wise fuzzy computation
h, w = arr.shape
fuzzy_output = np.full((h, w), np.nan, dtype=float)
for i in range(h):
    for j in range(w):
        if mask[i, j]:
            sim = ctrl.ControlSystemSimulation(quality_ctrl)
            sim.input['pm10'] = arr_norm[i, j]
            try:
                sim.compute()
                fuzzy_output[i, j] = sim.output['quality']
            except:
                fuzzy_output[i, j] = np.nan

# ------------------------------------------------------------
# Save TIFF
output_path = os.path.join(data_in_geoserver_folder, "zurich_fuzzy_quality.tif")
transform = from_bounds(bbox[0], bbox[1], bbox[2], bbox[3], w, h)

with rasterio.open(
    output_path,
    "w",
    driver="GTiff",
    height=h,
    width=w,
    count=1,
    dtype=fuzzy_output.dtype,
    crs=crs,
    transform=transform,
) as dst:
    dst.write(fuzzy_output, 1)

print(f"✅ Saved fuzzy environmental quality map: {output_path}")


✅ Raw PM10 TIFF downloaded: ../geoserver_data/data\zurich_pm10_raw.tif
✅ Saved fuzzy environmental quality map: ../geoserver_data/data\zurich_fuzzy_quality.tif
